<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=08963fca11a2ccbe1fbb6b592e0a0c45c89270c6c2567105325a7601b3c939fa
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-12 13:46:35
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 2.62 K (0.02%)
Current PnL: -28.61 L (-18.48%)
CY Booked + Current PnL: -14.32 L (-9.25%)
-------------------
Total profit:  1.33 L
Total loss:  -29.94 L
-------------------
Total Booked + Current PnL: 12.66 L (9.9%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.23%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.24 L (66.76%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.96%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.02,-16.79,20.29,0.09,17014.0,-16924.0,83853.0,94.66,42.0,M-SC,2.37,253.0,-0.99,0.62,10.16,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.02,-12.49,14.29,0.02,21829.0,-21803.0,152756.0,132.70,52.0,M-SC,15.82,234.0,-1.00,1.13,32.87,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.81,-1.24,23.58,22.05,22817.0,-1215.0,96765.0,-14.63,58.0,H-SC,6.96,164.0,-0.05,0.72,40.65,XR,ATH,FINANCE
82,VOLTAS,1530.00,1.00,6.93,11.93,19.69,24461.0,13293.0,205035.0,-3.56,52.0,X-MC,2.59,78.0,0.54,1.52,14.20,XY25,NTT,AC
43,ITC,452.00,-0.60,-2.15,12.86,10.44,25186.0,-4294.0,195844.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.30,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,4.32,-16.05,75.05,46.95,78941.0,-20112.0,105185.0,-22.59,40.0,H-SC,6.81,154.0,-0.25,0.78,31.77,MH,ATH,POWER
10,BANDHANBNK,400.00,3.10,-27.94,167.04,92.42,335281.0,-77840.0,200719.0,41.52,50.0,H-SC,8.72,170.0,-0.23,1.49,16.00,XY24,NTT,BANKS
22,DIXON,18931.72,2.88,-14.19,41.68,21.58,50124.0,-19881.0,120258.0,-56.73,36.0,X-MC,7.71,56.0,-0.40,0.89,8.19,X40N,ATH,IT
46,JSWINFRA,342.00,1.85,-8.38,25.76,15.23,47287.0,-16781.0,183566.0,-21.67,45.0,X-MC,8.88,66.0,-0.35,1.36,21.17,X40N,NTT,REALTY
67,SONACOMS,806.63,1.48,-15.18,64.52,39.54,55359.0,-15357.0,85802.0,-33.70,51.0,M-SC,4.61,220.0,-0.28,0.64,20.99,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.74,-17.32,51.82,25.51,79728.0,-32240.0,153855.0,-5.62,39.0,H-LC,4.12,49.0,-0.40,1.14,13.38,AR,ATH,ELECTRICAL
18,COFFEEDAY,80.00,-2.78,-45.06,146.23,35.27,91220.0,-51168.0,62381.0,-57.75,31.0,L-SC,11.08,270.0,-0.56,0.46,52.68,XR,NTT,HOTELS
57,RAJOOENG,143.10,-2.47,-38.27,126.17,39.61,79828.0,-39230.0,63270.0,-58.33,23.0,H-SC,28.09,135.0,-0.49,0.47,0.00,AR,ATH,MISC
26,GILLETTE,11206.78,-2.12,-4.22,38.25,32.42,93016.0,-10716.0,243180.0,-21.27,38.0,X-SC,9.17,85.0,-0.12,1.81,10.24,X40,ATH,FMCG
33,HINDUNILVR,2922.00,-2.03,-12.13,29.35,13.67,66299.0,-31173.0,225890.0,-18.78,25.0,X-LC,2.41,14.0,-0.47,1.68,6.64,XY25,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.60,-7.02,123.61,107.91,169688.0,-10367.0,137277.0,-24.20,38.0,M-SC,12.52,216.0,-0.06,1.02,2.36,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.02,-12.49,14.29,0.02,21829.0,-21803.0,152756.0,132.70,52.0,M-SC,15.82,234.0,-1.00,1.13,32.87,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.02,-16.79,20.29,0.09,17014.0,-16924.0,83853.0,94.66,42.0,M-SC,2.37,253.0,-0.99,0.62,10.16,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.14,-25.38,52.08,13.47,104792.0,-68454.0,201213.0,-69.28,31.0,H-SC,4.70,158.0,-0.65,1.49,2.31,XY24,NTT,PAINTS
66,SIS,528.00,-0.11,-25.58,62.94,21.26,52009.0,-28399.0,82633.0,1947.67,43.0,H-SC,4.79,166.0,-0.55,0.61,11.65,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.38,6.74,61.39,72.27,111034.0,11419.0,180867.0,-8.35,68.0,M-LC,2.02,99.0,0.1,1.34,13.33,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.38,6.74,61.39,72.27,111034.0,11419.0,180867.0,-8.35,68.0,M-LC,2.02,99.0,0.10,1.34,13.33,XR,NTT,IT
38,INDIAMART,4810.62,-0.77,-4.53,116.54,106.72,137220.0,-5591.0,117745.0,-52.73,30.0,H-SC,2.33,139.0,-0.04,0.87,17.78,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.75,-1.98,38.37,35.63,78820.0,-4154.0,205420.0,-15.85,43.0,H-MC,3.10,119.0,-0.05,1.52,14.36,AR,ATH,PHARMA
50,MASFIN,398.61,0.81,-1.24,23.58,22.05,22817.0,-1215.0,96765.0,-14.63,58.0,H-SC,6.96,164.0,-0.05,0.72,40.65,XR,ATH,FINANCE
25,FINCABLES,1641.55,-0.60,-7.02,123.61,107.91,169688.0,-10367.0,137277.0,-24.20,38.0,M-SC,12.52,216.0,-0.06,1.02,2.36,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.43,-21.63,89.42,48.45,140519.0,-43361.0,157145.0,-27.18,25.0,X-MC,12.19,64.0,-0.31,1.17,2.32,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-2.03,-12.13,29.35,13.67,66299.0,-31173.0,225890.0,-18.78,25.0,X-LC,2.41,14.0,-0.47,1.68,6.64,XY25,NTT,FMCG
75,TMPV,600.00,0.00,-17.86,73.09,42.18,165955.0,-49354.0,227056.0,-26.74,29.0,X-LC,6.38,3.0,-0.30,1.69,0.95,XY24,NTT,AUTO
23,DMART,5391.45,-0.18,-9.13,40.99,28.12,78373.0,-19207.0,191200.0,-21.58,31.0,X-LC,5.23,19.0,-0.25,1.42,12.34,X40N,ATH,FMCG
53,PAGEIND,51605.08,-0.36,-9.78,39.74,26.07,58704.0,-16020.0,147720.0,-33.51,32.0,X-MC,12.41,55.0,-0.27,1.10,0.64,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,-0.38,-80.25,531.73,24.79,264541.0,-202105.0,49751.0,-81.09,44.0,X-SC,0.92,86.0,-0.76,0.37,21.62,X40N,NTT,MISC
43,ITC,452.0,-0.60,-2.15,12.86,10.44,25186.0,-4294.0,195844.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.30,X40,NTT,FMCG
8,AWL,485.0,0.24,-21.51,95.56,53.50,226324.0,-64911.0,236840.0,-61.97,34.0,X-MC,2.09,58.0,-0.29,1.76,5.20,XY24,NTT,FMCG
36,ICICIPRULI,790.0,1.01,6.47,23.01,30.97,43300.0,11441.0,188179.0,-18.62,62.0,X-MC,2.12,75.0,0.26,1.40,19.85,X40,ATH,INSURANCE
3,ACC,3906.0,-0.08,-25.30,119.76,64.15,212861.0,-60211.0,177740.0,-56.31,34.0,X-SC,2.27,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.08,-25.30,119.76,64.15,212861.0,-60211.0,177740.0,-56.31,34.0,X-SC,2.27,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT
54,PGHH,17907.65,0.31,-5.88,42.11,33.76,79594.0,-11805.0,189015.0,-33.58,35.0,X-MC,5.35,57.0,-0.15,1.40,0.31,X40,ATH,FMCG
53,PAGEIND,51605.08,-0.36,-9.78,39.74,26.07,58704.0,-16020.0,147720.0,-33.51,32.0,X-MC,12.41,55.0,-0.27,1.10,0.64,X40,ATH,APPARELS
75,TMPV,600.00,0.00,-17.86,73.09,42.18,165955.0,-49354.0,227056.0,-26.74,29.0,X-LC,6.38,3.0,-0.30,1.69,0.95,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.03,-38.10,118.41,35.20,94315.0,-49019.0,79651.0,4.40,34.0,X-SC,15.90,92.0,-0.52,0.59,1.18,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.41,-29.57,102.80,42.84,47068.0,-19220.0,45786.0,-53.72,42.0,X-SC,36.19,83.0,-0.41,0.34,3.59,XY24,NTT,MISC
15,CAMS,4762.00,-0.38,-80.25,531.73,24.79,264541.0,-202105.0,49751.0,-81.09,44.0,X-SC,0.92,86.0,-0.76,0.37,21.62,X40N,NTT,MISC
59,RELAXO,1176.00,-0.16,-47.41,189.62,52.31,144750.0,-68823.0,76337.0,-44.05,43.0,X-SC,6.65,91.0,-0.48,0.57,2.18,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.03,-38.10,118.41,35.20,94315.0,-49019.0,79651.0,4.40,34.0,X-SC,15.90,92.0,-0.52,0.59,1.18,X40,NTT,FOOTWEAR
22,DIXON,18931.72,2.88,-14.19,41.68,21.58,50124.0,-19881.0,120258.0,-56.73,36.0,X-MC,7.71,56.0,-0.40,0.89,8.19,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,0.59,-12.76,36.73,19.28,110853.0,-44150.0,301806.0,-23.95,62.0,X-LC,4.93,1.0,-0.40,2.24,11.57,X40,ATH,IT
41,INFY,2275.00,-0.28,8.48,42.77,54.87,145168.0,26528.0,339416.0,-15.65,61.0,X-LC,3.31,2.0,0.18,2.52,17.46,X40,BTT,IT
75,TMPV,600.00,0.00,-17.86,73.09,42.18,165955.0,-49354.0,227056.0,-26.74,29.0,X-LC,6.38,3.0,-0.30,1.69,0.95,XY24,NTT,AUTO
81,VBL,671.64,1.17,-3.34,40.36,35.67,123212.0,-10559.0,305283.0,-14.83,57.0,X-LC,4.58,4.0,-0.09,2.27,10.05,X40N,ATH,FMCG
43,ITC,452.00,-0.60,-2.15,12.86,10.44,25186.0,-4294.0,195844.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.30,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.18,-37.59,112.92,32.89,54414.0,-29018.0,48188.0,-695.06,54.0,L-MC,5.86,259.0,-0.53,0.36,32.71,XR,NTT,BANKS
6,ASIANTILES,137.00,1.21,-14.88,113.43,81.67,90576.0,-13958.0,79852.0,7177.78,50.0,L-SC,13.62,271.0,-0.15,0.59,54.64,XR,NTT,CERAMICS
50,MASFIN,398.61,0.81,-1.24,23.58,22.05,22817.0,-1215.0,96765.0,-14.63,58.0,H-SC,6.96,164.0,-0.05,0.72,40.65,XR,ATH,FINANCE
13,BSOFT,831.70,0.26,-22.21,93.85,50.79,102679.0,-31245.0,109408.0,-0.27,67.0,H-SC,5.96,151.0,-0.30,0.81,27.39,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.02,-12.49,14.29,0.02,21829.0,-21803.0,152756.0,132.70,52.0,M-SC,15.82,234.0,-1.00,1.13,32.87,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,0.26,-22.21,93.85,50.79,102679.0,-31245.0,109408.0,-0.27,67.0,H-SC,5.96,151.0,-0.30,0.81,27.39,XR,ATH,IT
1,ABB,7934.0,0.72,-3.12,50.25,45.57,127366.0,-8155.0,253464.0,-37.78,60.0,H-MC,4.78,120.0,-0.06,1.88,7.85,AR,NTT,ELECTRICAL
84,WIPRO,420.0,0.38,6.74,61.39,72.27,111034.0,11419.0,180867.0,-8.35,68.0,M-LC,2.02,99.0,0.10,1.34,13.33,XR,NTT,IT
0,5PAISA,593.0,-0.13,-41.53,93.10,12.90,118939.0,-90742.0,127754.0,88.70,43.0,H-SC,11.85,161.0,-0.76,0.95,6.23,OX40N,NTT,FINANCE
56,RAJESHEXPO,518.0,-0.73,-63.10,171.18,0.07,86651.0,-86557.0,50620.0,1754.80,56.0,L-SC,2.41,268.0,-1.00,0.38,25.89,OX40N,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.82
1,25,45.73
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.06
MC    30.34
LC    24.61
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.34
X40      22.37
X40N     10.90
XR       10.13
XY25      9.39
AR        9.32
OX40N     7.76
SR        1.02
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.62
X-MC    23.55
X-LC    21.08
M-SC    12.09
X-SC     6.92
H-MC     5.03
L-SC     1.43
M-MC     1.40
M-LC     1.34
H-LC     1.14
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.31,-7.98,42.90
IT,12.81,-16.78,79.91
FINANCE,9.63,-19.58,70.10
ELECTRICAL,6.22,-11.95,52.95
MISC,6.06,-58.98,124.10
PAINTS,6.04,-13.33,30.16
INSURANCE,4.61,-1.95,37.88
PHARMA,4.10,-0.96,33.27
AUTO,3.44,-24.42,77.77


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3219914.0,21
AR,1314820.0,10
XR,1296095.0,13
X40,1025903.0,14
X40N,981675.0,9
OX40N,747365.0,10
XY25,376241.0,6
SR,283114.0,2
MH,78941.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3660738.0,25
M-SC,1430116.0,15
X-MC,1309849.0,16
X-SC,989331.0,8
X-LC,925665.0,11
H-MC,400846.0,3
L-SC,268447.0,3
M-LC,111034.0,1
H-LC,79728.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1274538.0      6
           AR         902316.0      5
M-SC       XY24       815790.0      6
H-SC       XR         774891.0      7
X-SC       X40N       507022.0      3
X-MC       X40        455494.0      7
           XY24       399512.0      3
X-LC       X40        383078.0      5
H-SC       OX40N      346938.0      4
M-SC       OX40N      313776.0      5
X-SC       XY24       294978.0      3
H-SC       SR         283114.0      2
X-MC       X40N       273068.0      4
X-LC       XY24       240436.0      2
H-MC       AR         206186.0      2
X-LC       X40N       201585.0      2
H-MC       XY24       194660.0      1
X-SC       X40        187331.0      2
L-SC       XR         181796.0      2
X-MC       XY25       181775.0      2
M-SC       XR         173960.0      2
           AR         126590.0      2
M-LC       XR         111034.0      1
X-LC       XY25       100566.0      2
L-SC       OX40N       86651.0      1
H-LC       AR          79728.0      1
H-SC       MH          78941.0      1
M-MC       XY25        56187.0      1
L-MC       XR          54414.0      1
L-LC       XY25        37713.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
